### Persistence
Using any open source model, we can program it to respond and remember us in a persistent manner, with help of states available in langchain framework

In [37]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
#db
# from langgraph.checkpoint.sqlite import SqliteSaver
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Before answering, analyze the user's question and what it means. For closed questions, answer swiftly and sharply. For open questions, provide appreciation and end with a follow up question"),
        MessagesPlaceholder(variable_name="messages")
    ]
  )
  
model = ChatOllama(
    model="llama3.2:latest",
    temperature=0.8, 
    top_p=0.95,
    top_k=50,
    num_ctx=2048, 
    repeat_penalty=1.0
)
#Define a new graph
workflow = StateGraph(state_schema=MessagesState)
#define a function that calls model
def call_model(state: MessagesState):
  prompt = prompt_template.invoke(dict(state))
  response = model.invoke(prompt)
  return {"messages" : response}

#define a node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
#Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
# Two different threads representing two different conversations
# Savign memory and remembering the users will be quite beneficial for the model
config = {"configurable" : {"thread_id" : "1"}}
config_2 = {"configurable" : {"thread_id" : "2"}}

# This function is for streaming the output of the model
def stream_output(app=app, query="", config=config):
  input_messages = [HumanMessage(content=query)]
  for chunk, metadata in app.stream({"messages": input_messages}, config=config, stream_mode="messages"):
    if isinstance(chunk, AIMessage):
      print(chunk.content, end = "", flush=True)

## Here we told him our name

In [38]:
query = "My name is Deepesh"
stream_output(app, query=query, config=config)

Nice to meet you, Deepesh! Is there something I can help you with, or would you like to chat for a bit?

## He remembers it

In [39]:
query = "What's my name?"
stream_output(app, query=query, config=config)

Your name is already mentioned, Deepesh!

### Using different ID here, the model forgets who we are
Since altering thread ID, the model now forgets the previous chats

In [40]:
query = "What's my name?"
stream_output(app, query=query, config=config_2)

I don't have any information about you, so I won't be able to provide your name. Can I help you with anything else?

### However, hook back to the previous thread id, and now the model again remembers your name

In [47]:
query = "What's my full Name?"
input_messages = [HumanMessage(query)]
stream_output(app, query=query, config=config)

I think we've reached a breaking point! Your full name is DEEPESH DHAKAL, and I've got it memorized. No need to ask again, my friend!

In [45]:
query = "My full name is Deepesh Dhakal"
input_messages = [HumanMessage(query)]
stream_output(app, query=query, config=config)

I think we've covered this ground quite thoroughly! Yes, your full name is indeed Deepesh Dhakal. Would you like to talk about something else, or is there a particular topic you'd like to explore?

In [44]:
query = "Great, what's my full name?"
input_messages = [HumanMessage(query)]
stream_output(app, query=query, config=config)

Déjà vu again! Still Deepesh Dhakal, my friend! Would you like to explore something else or test my naming skills again?